In [2]:
import bpy
import bmesh
import mathutils as bm

/run/user/1000/gvfs/ non-existent directory


In [2]:
def v(*p): return bm.Vector(p)

In [3]:
class box:
    def __init__(self):
        self.min = None
        self.max = None

    def add(self, p):
        if self.min is None or self.max is None:
            self.min = p
            self.max = p
            return self
        self.min = bm.Vector((min(x, y) for x,y in zip(self.min, p)))
        self.max = bm.Vector((max(x, y) for x,y in zip(self.max, p)))
        return self
    def size(self):
        return self.max - self.min
    def radius(self):
        return self.size().length
    def center(self): 
        return (self.max + self.min) * 0.5

    def create_box(self, mesh, margin=0):
        m = bm.Matrix.LocRotScale(self.center(), None, self.size() + bm.Vector.Fill(3, 2 * margin))
        
        r = bmesh.ops.create_cube(mesh, size=1, matrix=m)

        mesh.verts.ensure_lookup_table()
        mesh.edges.ensure_lookup_table()
        mesh.faces.ensure_lookup_table()
        return r

    @staticmethod
    def bbox(mesh):
        b = box()
        for i in mesh.verts[:]:
            b.add(i.co)
        return b

In [203]:
m = bmesh.new()
box().add(v(0, 0, 0)).add(v(1,1,1)).create_box(m)

{'verts': [<BMVert(0x563921bce9b0), index=0>,
  <BMVert(0x563921bce9e8), index=1>,
  <BMVert(0x563921bcea20), index=2>,
  <BMVert(0x563921bcea58), index=3>,
  <BMVert(0x563921bcea90), index=4>,
  <BMVert(0x563921bceac8), index=5>,
  <BMVert(0x563921bceb00), index=6>,
  <BMVert(0x563921bceb38), index=7>]}

In [240]:
vv = volume.create(m, None, flags=1)

In [253]:
def i(x):
    print(vv.bounds[x].projection, vv.bounds[x].displacement)
    print(vv.bounds[x].sample(bm.Vector((1, 1))))
    for j,f in zip(vv.bounds[x].bounds, m.faces[x].edges[:]):
        print('-----')
        print([y.co for y in f.verts[:]])
        print(j.sample(j.bounds[0]), j.sample(j.bounds[1]))
        print(vv.bounds[x].sample(j.sample(j.bounds[0])))
        print(vv.bounds[x].sample(j.sample(j.bounds[1])))

In [11]:
None == False

False

In [179]:
print(p0 @ v(1, 0, 0))
print(p0 @ v(0, 1, 0))

<Vector (0.0000, -0.6667, 0.5774)>
<Vector (0.7071, 0.3333, 0.5774)>


In [52]:
class __projection:
    v0 = bm.Vector((0, 1, -1))
    v1 = bm.Vector((1, 0, -1))
    v2 = bm.Vector((-1, 1))

def build_projection(normal) -> bm.Matrix:
    m = bm.Matrix()
    # normal assumed to be normalized
    d = len(normal)
    if d == 3:
        if normal[0] * normal[0] < 0.5: # close enough to x=0 plane
            t = normal.xzy * __projection.v0 # bm.Vector((0, normal[2], -normal[1]))
        else:
            t = normal.zyx * __projection.v1 # bm.Vector((normal[2], 0, -normal[0]))
        m[0][:3] = t.normalized()[:]
        m[1][:3] = normal.cross(t)[:]
        m[2][:3] = normal.xyz
        return m.to_3x3()
    if d == 2:
        m[0][:2] = normal.yx * __projection.v2 # bm.Vector(-normal[1], normal[0]).normalized()
        m[1][:2] = normal.xy
        return m.to_2x2()
    if d == 1:
        return normal
    return None

def get_plane(plane):
    d = len(plane) - 1
    n = bm.Vector(plane[:d]).length
    return bm.Vector(plane) / n

class volume:
    def __init__(self, projection, displacement, flags):
        self.projection = projection
        self.displacement = displacement
        self.flags = flags

    def set_bounds(self, bounds):
        self.bounds = bounds
        return self

    def project_vector(self, vector):
        d = len(self.projection)
        return (self.projection @ vector.resized(d))

    def sample(self, pos):
        d = len(self.projection)
        if d == 2:
            return bm.Vector((pos, self.displacement)) @ self.projection
        v = pos.resized(d)
        v[d - 1] = self.displacement
        return (v @ self.projection)

    @staticmethod
    def create(bmesh_obj, parent: 'volume', flags_layers=(None, None, None), flags=0):
        if isinstance(bmesh_obj, bmesh.types.BMesh):
            flags_layers = (bmesh_obj.faces.layers.int.get("flags"), bmesh_obj.edges.layers.int.get("flags"), bmesh_obj.verts.layers.int.get("flags"))
            v = volume(bm.Matrix.Identity(4), 0, flags)
            return v.set_bounds([volume.create(f, v, flags_layers) for f in bmesh_obj.faces[:]])
        if isinstance(bmesh_obj, bmesh.types.BMFace):
            projection = build_projection(bmesh_obj.normal.normalized())
            displacement = projection[2] @ bmesh_obj.verts[0].co
            flags = bmesh_obj[flags_layers[0]] if flags_layers[0] else 1
            v = volume(projection, displacement, flags)
            return v.set_bounds([volume.create(e, v, flags_layers) for e in bmesh_obj.edges[:]])
        if isinstance(bmesh_obj, bmesh.types.BMEdge):
            p0 = parent.project_vector(bmesh_obj.verts[0].co)
            p1 = parent.project_vector(bmesh_obj.verts[1].co)
            normal = (-(p1 - p0).yx * __projection.v2).normalized()
            projection = build_projection(normal)
            displacement = p0.xy @ normal
            
            flags = bmesh_obj[flags_layers[1]] if flags_layers[1] else 1

            v = volume(projection, displacement, flags)
            return v.set_bounds([projection[0] @ p0.xy, projection[0] @ p1.xy])

def union(x,y): return x | y
def intersect(x,y): return x & y
def symdiff(x,y): return x ^ y

from abc import abstractmethod
from typing import Type, cast

class bsp_node_base:
    def __init__(self, parent: 'bsp_node_nd | None', dimension, index=-1):
        self.dimension = dimension
        self.parent = parent
        self.index = index

    def is_root(self): return self.parent is None
    @abstractmethod
    def instance(self, parent: 'bsp_node_nd | None'=None) -> 'bsp_node_base': pass
    @abstractmethod
    def copy(self, parent: 'bsp_node_nd | None') -> 'bsp_node_base': pass
    # @abstractmethod
    # def is_leaf(self) -> bool: return isinstance(self, bsp_leaf)
    @abstractmethod
    def inverse(self) -> 'bsp_node_base': pass
    @abstractmethod
    def add(self, node: 'bsp_node_nd', cut=False) -> 'bsp_node_nd': pass
    @abstractmethod
    def optimize(self) -> 'bsp_node_base': pass
    @abstractmethod
    def csg(self, tree, operation=union) -> 'bsp_node_base': pass
    @abstractmethod
    def print(self): pass
    def is_empty(self): return False

    def split(self, node, _is_front): return self
    def numerate(self):
        nodes = []
        index = 0
        branches = []
        leafs = []
        stack = [self]
        while len(stack) > 0:
            n = stack.pop()
            nodes.append(n)
            n.index = index
            index += 1
            if isinstance(n, bsp_node_nd):
                stack.append(n._f)
                stack.append(n._b)
                branches.append(n.index)
            else:
                leafs.append(n.index)
        return nodes, branches, leafs

class bsp_leaf(bsp_node_base):
    def __init__(self, parent: 'bsp_node_nd | None', dimension, flags = 0, index=-1):
        super(bsp_leaf, self).__init__(parent, dimension, index)
        self.flags = flags
    
    def copy(self, parent): return self.instance(parent)
    def instance(self, parent: 'bsp_node_nd | None'=None, save_index=False): 
        return bsp_leaf(parent=parent, dimension=self.dimension, flags=self.flags, index=self.index if save_index else -1)

    def add(self, node: 'bsp_node_nd', cut_front=None) -> 'bsp_node_nd':
        res = node.instance(self.parent)
        res.set_f(self.instance())
        res.set_b(self.instance())
        if cut_front == True:
            cast(bsp_leaf, res._f).flags = 0
        if cut_front == False:
            cast(bsp_leaf, res._b).flags = 0
        return res
    
    def csg(self, tree: 'bsp_node_base', operation=union):
        if isinstance(tree, bsp_leaf):
            self.flags = operation(self.flags, tree.flags)
            return self
        else:
            assert(isinstance(tree, bsp_node_nd))
            s = [tree]
            while len(s)>0:
                n = s.pop()
                if isinstance(n, bsp_leaf):
                    n.flags = operation(self.flags, n.flags)
                else:
                    s.append(n._b)
                    s.append(n._f)
            return tree

    def is_empty(self): return self.flags == 0
    def inverse(self):
        res = self.instance(self.parent)
        res.flags = ~res.flags
        return res

    def divide(self, splitter: 'bsp_node_nd'):
        if self.is_empty():
            return self
        return (
            splitter.instance(self.parent).with_back(self.instance()),
            splitter.instance(self.parent).with_front(self.instance())
        )

    def print(self):
        print(f'leaf<{self.dimension}> {self.index}: {self.flags}')
    def optimize(self) -> 'bsp_node_base': return self

class bsp_node_nd(bsp_node_base):
    def __init__(self, parent, dimension, index=-1):
        super(bsp_node_nd, self).__init__(parent, dimension, index)
        self._f: bsp_node_base = bsp_leaf(self, dimension, 0)
        self._b: bsp_node_base = bsp_leaf(self, dimension, 0)

    @abstractmethod
    def instance(self, parent) -> 'bsp_node_nd': pass
    def split(self, _splitter, _is_front) -> 'bsp_node_nd': raise NotImplemented()
    def is_root(self): return self.parent is None or self.parent.dimension == self.dimension + 1

    def with_front(self, front):
        self.set_f(front)
        return self
    def with_back(self, back):
        self.set_b(back)
        return self

    def copy(self, parent):
        r = self.instance(parent)
        r.set_f(self._f.copy(None))
        r.set_b(self._b.copy(None))
        return r

    def csg(self, tree: 'bsp_node_base', operation=union):
        res = tree.add(self)
        self.set_b(self._b.csg(res._b, operation))
        self.set_f(self._f.csg(res._f, operation))
        return self
        # if isinstance(tree, bsp_node_nd):
        # else:
        #     self.set_f(self._f.csg(tree, operation))
        #     self.set_b(self._b.csg(tree, operation))
        #     return self

    def set_f(self, node):
        self._f = node
        node.parent = self
    def set_b(self, node):
        self._b = node
        node.parent = self

    def inverse(self):
        res = self.instance(self.parent)
        res.set_f(self._f.inverse())
        res.set_b(self._b.inverse())
        return res

    def divide(self, splitter: 'bsp_node_nd'):
        pass

    def merge(self, tree: 'bsp_node_nd'):
        order = set()
        ind = 0
        stack = [tree]
        while len(stack) > 0:
            n = stack.pop()
            n.index = ind
            ind += 1
            if isinstance(n, bsp_node_nd):
                t = n.parent
                while t is not None:
                    order.add((n.index, t.index))
                    t = t.parent
                stack.append(n._b)
                stack.append(n._f)

        
        



    def add(self, node: 'bsp_node_nd', cut_front=None):
        res = node.instance(self.parent)
        if cut_front is None:
            # place entire tree into both child nodes (with copy)
            cp = self.copy(self.parent).split(node, False)
            res.set_f(self.split(node, True))
            res.set_b(cp)
        elif cut_front == True:
            res.set_b(self.split(node, False))
        elif cut_front == False:
            res.set_f(self.split(node, True))
        return res
        
    def optimize(self) -> 'bsp_node_base':
        self.set_f(self._f.optimize())
        self.set_b(self._b.optimize())

        if isinstance(self._f, bsp_leaf) and isinstance(self._b, bsp_leaf) and self._f.flags == self._b.flags:
            return self._f
        return self

class bsp_node_1d(bsp_node_nd):
    def __init__(self, side, displacement, parent):
        super(bsp_node_1d, self).__init__(parent, 1)
        self.side = side
        self.displacement = displacement

    def classify(self, node: 'bsp_node_1d'):
        nn = self.side * node.side
        dd = nn * self.displacement - node.displacement
        res = 0x0 # incident
        if dd > 1e-3: res = 0x1 # front
        elif dd < -1e-3: res = 0x2 # back
        if nn > 0.0: return res
        return res | 0x4 # opposite
    # def classify(self, node):
    #     nn = self.side * node.side
    #     dd = self.displacement - nn * node.displacement
    #     if dd < -1e-3: return 0x1 # front
    #     if dd >  1e-3: return 0x2 # back
    #     if nn > 0.0: return 0x0 # incident
    #     return 0x4 # opposite
            
    def instance(self, parent):
        return bsp_node_1d(self.side, self.displacement, parent)

    def split(self, node: 'bsp_node_1d', is_front):
        clsf = self.classify(node)

        if not is_front and clsf & 0x1: return self._f if clsf & 0x4 else self._b
        if is_front and clsf & 0x2: return self._b if clsf & 0x4 else self._f
        if (clsf & 0x3) == 0: return self._f if is_front ^ bool(clsf & 0x4) else self._b
        # if clsf == 0x4: return self._b if is_front else self._f
        self.set_f(self._f.split(node, is_front))
        self.set_b(self._b.split(node, is_front))
        return self

        if   clsf == 0x1: self.set_f(self._f._add(node, is_front))
        elif clsf == 0x2: self.set_b(self._b._add(node, is_front))
        else: # incident (flip side if it's opposite)
            return self._f if is_front ^ (clsf == 0x0) else self._b
        # if isinstance(self._f, bsp_leaf) and isinstance(self._b, bsp_leaf) and self._f.flags == self._b.flags:
        #     return bsp_leaf(None, self.dimension, self._f.flags)
        return self._f if is_front else self._b
    def print(self):
        print(f"node<1> {self.index}: {self.side} {self.displacement}\n  {self._b.index} {self._f.index}")
        self._b.print()
        self._f.print()

class bsp_node(bsp_node_nd):
    def __init__(self, projection, displacement, parent, bounds=None):
        super(bsp_node, self).__init__(parent, len(projection))
        self.projection = projection
        self.displacement = displacement
        self.bounds = bsp_leaf(None, self.dimension-1, 0) if bounds is None else bounds  # d-1 bsp_tree
        self.bounds.parent = self

    @staticmethod
    def create(vol: volume) -> 'bsp_node':
        raise NotImplemented()

    def instance(self, parent):
        return bsp_node(self.projection, self.displacement, parent, self.bounds.copy(None))

    def project(self, node):
        ndim = self.dimension - 1
        n = self.projection @ node.projection[ndim]
        d = node.displacement * (n @ n) - n[ndim] * self.displacement
        if ndim > 1:
            return bsp_node(build_projection(n.resized(ndim)), d, None)
        return bsp_node_1d(n[0], d, None)

    def classify(self, node):
        dim = self.dimension - 1

        nn = node.projection[dim] @ self.projection[dim]
        if abs(1.0 - abs(nn)) < 1e-4: # parallel (almost)
            dd = nn * self.displacement - node.displacement
            if dd > 1e-3: return 0x1 # front
            if dd < -1e-3: return 0x2 # back
            if nn > 0:
                return 0x0 # incident
            return 0x4 # opposite
        return 0x3 # crossing

    def split(self, splitter: 'bsp_node', is_front):
        assert(splitter.dimension == self.dimension)
        
        clsf = self.classify(splitter)
        if (clsf & 0x3) == 0: # incident
            # 'node' is already the root, so depending on which side 'self' is coherent modify boundary
            splitter.bounds = splitter.bounds.csg(self.bounds, union if clsf == 0x0 else symdiff)
            res = self._f if is_front ^ (clsf == 0x0) else self._b
            splitter.bounds = splitter.bounds.optimize()
            # skip node
            return res.split(splitter, is_front)

        bnd_splitter = self.project(splitter)
        bounds = self.bounds.add(bnd_splitter)
        if self.bounds.is_empty():
            return bsp_leaf(None, self.dimension, 0)
        self.set_f(self._f.split(splitter, is_front))
        self.set_b(self._b.split(splitter, is_front))
        # if isinstance(self._f, bsp_leaf) and isinstance(self._b, bsp_leaf) and self._f.flags == self._b.flags:
        #     return bsp_leaf(None, self.dimension, self._f.flags)
        return self
    def print(self):
        print(f"node<{self.dimension}> {self.index}: {self.projection} {self.displacement}\n  {self._f.index} {self._b.index}")
        self._f.print()
        self._b.print()

class bsp_graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes # {i: bsp_graph_node}
        self.edges = edges # {(i, j): portal}
    def __getitem__(self, idx):
        return self.nodes[idx]
    @staticmethod
    def build(tree: bsp_node_nd | bsp_leaf):
        # numerate
        nodes, _, leafs = tree.numerate()
        # use dynamic programming to build boundaries
        num_nodes = len(nodes)
        lcas = [[None for _ in range(0,num_nodes)] for _ in range(0,num_nodes)]
        bounds = [[None for _ in range(0,num_nodes)] for _ in range(0,num_nodes)]

        def lca(node: bsp_node_base, table):
            if isinstance(node, bsp_node_nd):
                ff = list(lca(node._f, table)) + [node._f]
                bb = list(lca(node._b, table)) + [node._b]
                for i in ff:
                    for j in bb:
                        table[i.index][j.index] = node
                        table[j.index][i.index] = node
                for i in ff:
                    table[node.index][i.index] = node
                    table[i.index][node.index] = node
                    yield i
                for i in bb:
                    table[node.index][i.index] = node
                    table[i.index][node.index] = node
                    yield i

        _ = list(lca(tree, lcas))

        def bound(n0: bsp_node_base, n1: bsp_node_base, lcas, bounds):
            if bounds[n0.index][n1.index] is not None:
                return bounds[n0.index][n1.index]
            
            lca = lcas[n0][n1]
            n0p = n0.parent if n0 != lca else None
            n1p = n1.parent if n1 != lca else None

            # if n0p

        num_leafs = len(leafs)
        for i in range(0, num_leafs):
            for j in range(i+1, num_leafs):
                pass




In [53]:
import operator
r0 = bsp_node_1d(1.0, 1.0, None)
r0._b.flags = 1
r1 = bsp_node_1d(-1.0, 0.0, None)
r1._b.flags = 1
s0 = r0.csg(r1, operator.__and__)

r0 = bsp_node_1d(1.0, 3.0, None)
r0._b.flags = 2
r1 = bsp_node_1d(-1.0, 4.0, None)
r1._b.flags = 2
s1 = r0.csg(r1, operator.__and__)

In [48]:
r0 = bsp_node_1d(1.0, 1.0, None)
r0._b.flags = 1
r1 = bsp_node_1d(-1.0, -2.0, None)
r1._b.flags = 2
s0 = r0.csg(r1, operator.__or__)
s0.numerate()
s0.print()

node<1> 0: 1.0 1.0
  1 2
leaf<1> 1: 1
node<1> 2: -1.0 -2.0
  3 4
leaf<1> 3: 2
leaf<1> 4: 0


node<1> 0: 1.0 1.0
  1 4
node<1> 1: -1.0 -1.0
  2 3
leaf<1> 2: 3
leaf<1> 3: 1
node<1> 4: -1.0 -1.0
  5 6
leaf<1> 5: 2
leaf<1> 6: 0


In [54]:
s = s0.csg(r0, operator.__or__)
s.numerate()

([<__main__.bsp_node_1d at 0x7f5dfa9b7130>,
 [0, 1, 3, 6],
 [2, 4, 5, 7, 8])

In [55]:
s.print()

node<1> 0: 1.0 1.0
  1 6
node<1> 1: -1.0 0.0
  2 3
leaf<1> 2: 3
node<1> 3: -1.0 4.0
  4 5
leaf<1> 4: 2
leaf<1> 5: 0
node<1> 6: 1.0 3.0
  7 8
leaf<1> 7: 2
leaf<1> 8: 0


In [169]:
s0._f

In [168]:
s0._b

In [61]:
planes = [(build_projection(bm.Vector((-1,  0,  0))), 0),
          (build_projection(bm.Vector(( 1,  0,  0))), 1),
          (build_projection(bm.Vector(( 0, -1,  0))), 0),
          (build_projection(bm.Vector(( 0,  1,  0))), 1),
          (build_projection(bm.Vector(( 0,  0, -1))), 0),
          (build_projection(bm.Vector(( 0,  0,  1))), 1)]

nodes = [bsp_node(x[0], x[1], None, bsp_leaf(None, 2, 1)) for x in planes]
for i in nodes:
    i._b.flags = 1
    i.print()

node<3> -1: <Matrix 3x3 ( 0.0000, 0.0000,  1.0000)
            ( 0.0000, 1.0000, -0.0000)
            (-1.0000, 0.0000,  0.0000)> 0
  -1 -1
leaf<3> -1: 0
leaf<3> -1: 1
node<3> -1: <Matrix 3x3 ( 0.0000, 0.0000, -1.0000)
            (-0.0000, 1.0000,  0.0000)
            ( 1.0000, 0.0000,  0.0000)> 1
  -1 -1
leaf<3> -1: 0
leaf<3> -1: 1
node<3> -1: <Matrix 3x3 ( 0.0000,  0.0000, 1.0000)
            (-1.0000,  0.0000, 0.0000)
            ( 0.0000, -1.0000, 0.0000)> 0
  -1 -1
leaf<3> -1: 0
leaf<3> -1: 1
node<3> -1: <Matrix 3x3 ( 0.0000, 0.0000, -1.0000)
            (-1.0000, 0.0000,  0.0000)
            ( 0.0000, 1.0000,  0.0000)> 1
  -1 -1
leaf<3> -1: 0
leaf<3> -1: 1
node<3> -1: <Matrix 3x3 ( 0.0000, -1.0000, -0.0000)
            (-1.0000,  0.0000, -0.0000)
            ( 0.0000,  0.0000, -1.0000)> 0
  -1 -1
leaf<3> -1: 0
leaf<3> -1: 1
node<3> -1: <Matrix 3x3 ( 0.0000, 1.0000, -0.0000)
            (-1.0000, 0.0000,  0.0000)
            ( 0.0000, 0.0000,  1.0000)> 1
  -1 -1
leaf<3> -1: 0
lea

In [64]:
root = nodes[0]
for i in nodes[1:]:
    root = root.csg(i, intersect)

root.numerate()

([<__main__.bsp_node at 0x7f55e432eb90>,
 [0],
 [1, 2])

In [46]:
root.print()

node<3> 0: <Matrix 3x3 ( 0.0000, 1.0000, -0.0000)
            (-1.0000, 0.0000,  0.0000)
            ( 0.0000, 0.0000,  1.0000)> 1
  2 1
node<3> 2: <Matrix 3x3 ( 0.0000, -1.0000, -0.0000)
            (-1.0000,  0.0000, -0.0000)
            ( 0.0000,  0.0000, -1.0000)> 0
  4 4
leaf<3> 4: 0
leaf<3> 4: 0
leaf<3> 1: 0


In [8]:
i = [0]
r = bt(bt(bt(bt.leaf(i), bt.leaf(i), i), bt.leaf(i), i), bt(bt(bt(bt.leaf(i), bt.leaf(i), i), bt(bt.leaf(i), bt.leaf(i), i), i), bt.leaf(i), i), i)
md = r.depth()
r.print(md)

14 4 13
4 2 3
13 11 12
2 0 1
11 7 10
7 5 6
10 8 9


In [10]:
table = [[-1 for _ in range(i[0])] for _ in range(i[0])]
def print_table(t):
    for i in t:
        print(' '.join([f"{x:2}" for x in i]))
ops = [0]
print([x.i for x in list(r.lca(table, ops))])
print_table(table)
print(ops[0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
-1  2  2  4  4 14 14 14 14 14 14 14 14 14 14
 2 -1  2  4  4 14 14 14 14 14 14 14 14 14 14
 2  2 -1  4  4 14 14 14 14 14 14 14 14 14 14
 4  4  4 -1  4 14 14 14 14 14 14 14 14 14 14
 4  4  4  4 -1 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 -1  7  7 11 11 11 11 13 13 14
14 14 14 14 14  7 -1  7 11 11 11 11 13 13 14
14 14 14 14 14  7  7 -1 11 11 11 11 13 13 14
14 14 14 14 14 11 11 11 -1 10 10 11 13 13 14
14 14 14 14 14 11 11 11 10 -1 10 11 13 13 14
14 14 14 14 14 11 11 11 10 10 -1 11 13 13 14
14 14 14 14 14 11 11 11 11 11 11 -1 13 13 14
14 14 14 14 14 13 13 13 13 13 13 13 -1 13 14
14 14 14 14 14 13 13 13 13 13 13 13 13 -1 14
14 14 14 14 14 14 14 14 14 14 14 14 14 14 -1
105


In [65]:
i[0] * (i[0] - 1) / 2

105.0

In [14]:
import math
class meshutils:
    eps = 1e-5
    @staticmethod
    def get_tangents(plane):
        if plane.x * plane.x < 0.5: # close enough to x=0 plane
            t = bm.Vector((0, plane.z, -plane.y))
        else:
            t = bm.Vector((plane.z, 0, -plane.x))
        t.normalize()
        
        return t, t.cross(plane.xyz)

    @staticmethod
    def cut_segment(segment, plane):
        pass

    @staticmethod
    def classify_point(plane, point):
        d = (plane.xyz @ point) - plane.w
        if d > meshutils.eps:
            return 0x1 # front
        if d < -meshutils.eps:
            return 0x2 # back
        return 0x0 # in plane

    @staticmethod
    def point_positive(plane, point):
        return ((plane.xyz @ point) - plane.w) > 0
        
    @staticmethod
    def classify_face(plane, face):
        res = 0
        for v in face.verts[:]:
            c = meshutils.classify_point(plane, v.co)
            res |= c
            if res == 0x3: # crossing
                break
        return res

    @staticmethod
    def get_plane(face):
        return bm.Vector((*face.normal, face.normal @ face.verts[0].co))

    @staticmethod
    def slice_mesh(mesh, plane, clean_outer=True):
        bmesh.ops.bisect_plane(mesh, geom=mesh.faces[:] + mesh.edges[:] + mesh.verts[:], 
            plane_co=plane.xyz * plane.w, plane_no=plane.xyz, clear_inner=not clean_outer, clean_outer=clean_outer)
        bmesh.ops.holes_fill(mesh, edges=mesh.edges, sides=len(mesh.edges))
        bmesh.ops.delete(mesh, geom=[x for x in mesh.faces if x.calc_area() < meshutils.eps], context='FACES')
        for face in mesh.faces:
            bmesh.ops.remove_doubles(mesh, verts=face.verts[:], dist=meshutils.eps)
    @staticmethod
    def split(mesh, plane):
        # suboptimal (probably need to split before clean)
        front = mesh.copy()
        return (meshutils.slice_mesh(front, plane, False), meshutils.slice_mesh(mesh, plane, True))


In [7]:
bm.geometry.points_in_planes([
    bm.Vector(( 1, 0, 0, 1)),
    bm.Vector(( 0, 1, 0, 1)),
    bm.Vector(( 0, 0, 1, 1)),
])

([Vector((-1.0, -1.0, -1.0))], [0, 1, 2])

In [ ]:
from enum import IntFlag
import qsl

class bsp_face_build_flags(IntFlag):
    hint = qsl.Q_SURF_HINT # preferable splitter
    areaportal = qsl.Q_CONT_AREAPORTAL # separates areas
    translucent = qsl.Q_CONT_TRANSLUCENT # has geometry below (doesn't split areas)
    nodraw = qsl.Q_SURF_NODRAW # doesn't present as face
class bsp_face_flags(IntFlag):
    slick = qsl.Q_SURF_SLICK
    noimpact = qsl.Q_SURF_NOIMPACT
    nomarks = qsl.Q_SURF_NOMARKS
    nodamage = qsl.Q_SURF_NODAMAGE
    noob = qsl.Q_SURF_NOOB
    metalsteps = qsl.Q_SURF_METALSTEPS
    flesh = qsl.Q_SURF_FLESH
    nosteps = qsl.Q_SURF_NOSTEPS
    dust = qsl.Q_SURF_DUST
class bsp_content_flags(IntFlag): # affect brushes only
    trigger = qsl.Q_CONT_TRIGGER
    playerclip = qsl.Q_CONT_PLAYERCLIP
    water = qsl.Q_CONT_WATER
    slime = qsl.Q_CONT_SLIME
    lava = qsl.Q_CONT_LAVA
    nodrop = qsl.Q_CONT_NODROP
    fog = qsl.Q_CONT_FOG

class _bsp_sub:
    @classmethod
    def positive(cls, plane, point) -> bool:
        raise
    @classmethod
    def classify(cls, plane, face) -> int:
        raise
    @classmethod
    def split_volume(cls, vol, plane):
        return ([], [])
    @classmethod
    def is_adjacent(cls, vol, plane) -> bool:
        return False
    def __init__(self, faces): #, volume):
        self.plane = None
        self.faces = faces
        # self.volume = volume
        self.child_f = None
        self.child_b = None

        self.sides = []

    def split(self, plane):
        if len(self.faces) == 0:
            return
        self.plane = plane
        f = []
        b = []
        inc = []
        for i in self.faces:
            r = self.__class__.classify(plane, i)
            if r == 0:
                inc.append(i)
            else:
                if r & 0x1:
                    f.append(i)
                if r & 0x2:
                    b.append(i)
        
        fv, bv = self.__class__.split_volume(self.volume, plane)
        self.child_f = _bsp_sub(f, fv)
        self.child_b = _bsp_sub(b, bv)
        self.volume = None
        self.faces = inc
        yield self.child_f
        yield self.child_b

    def find(self, point):
        c = self.__class__.positive(self.plane, point)
        if c:
            if self.child_f is not None:
                return self.child_f.find(point)
            return self
        if self.child_b is not None:
            return self.child_b.find(point)
        return self

    

class bsp_sub_3d(_bsp_sub):
    @classmethod
    def positive(cls, plane, point) -> bool:
        return meshutils.point_positive(plane, point)
    @classmethod
    def classify(cls, plane, face) -> int:
        return meshutils.classify_face(plane, face)
    @classmethod
    def split_volume(cls, vol, plane):
        return meshutils.split(vol, plane)

class bsp_tree:
    def __init__(self, mesh, flags: bsp_content_flags, mode='SUBTRACTIVE'):
        self.mode = mode
        self.mesh = bmesh.new()
        self.mesh.from_mesh(mesh)
        self.flags = flags
        self._build_flags_layer = self.mesh.faces.layers.int("face_build_flags")
        self._flags_layer = self.mesh.faces.layers.int("face_flags")


    def _build_tree(self):
        if len(self.faces) == 0:
            return None
        self.volume = box.bbox(self.mesh).create_box(m, 16)
        self.faces = list(self.mesh.faces[:])
        self.plane = None
        # find best splitter

    
    def _pick_splitter(self, faces):
        hint = None
        areaportal = None
        for i in self.faces:
            if self._get_build_flag(i, bsp_face_build_flags.hint):
                hint = i
                break
            if not areaportal and self._get_build_flag(i, bsp_face_build_flags.hint):
                areaportal = i
        if not hint:
            hint = areaportal if areaportal else self.faces[0]
        p = meshutils.get_plane(hint)



    def _get_build_flag(self, face, flag):
        pass

In [114]:
q = bm.Vector(( 1, 0, 0, 1))
q.to_4d()

Vector((1.0, 0.0, 0.0, 1.0))

In [3]:
m = bmesh.new()

In [4]:
bmesh.ops.create_cube(m, size=10, matrix=bm.Matrix.LocRotScale(bm.Vector((1, 0, 0)), None, None))
[x.co for x in m.verts]

[Vector((-4.0, -5.0, -5.0)),
 Vector((-4.0, -5.0, 5.0)),
 Vector((-4.0, 5.0, -5.0)),
 Vector((-4.0, 5.0, 5.0)),
 Vector((6.0, -5.0, -5.0)),
 Vector((6.0, -5.0, 5.0)),
 Vector((6.0, 5.0, -5.0)),
 Vector((6.0, 5.0, 5.0))]

In [6]:
m.faces.ensure_lookup_table()

In [81]:
bb = box.bbox(m)

In [82]:
bb.radius()

17.320508075688775

In [42]:
u = bmesh.new()
bb.create_box(u, 5)
u.verts.ensure_lookup_table()
[x.co for x in u.verts[:]]

<Matrix 4x4 (20.0000,  0.0000,  0.0000, 1.0000)
            ( 0.0000, 20.0000,  0.0000, 0.0000)
            ( 0.0000,  0.0000, 20.0000, 0.0000)
            ( 0.0000,  0.0000,  0.0000, 1.0000)>


[Vector((-9.0, -10.0, -10.0)),
 Vector((-9.0, -10.0, 10.0)),
 Vector((-9.0, 10.0, -10.0)),
 Vector((-9.0, 10.0, 10.0)),
 Vector((11.0, -10.0, -10.0)),
 Vector((11.0, -10.0, 10.0)),
 Vector((11.0, 10.0, -10.0)),
 Vector((11.0, 10.0, 10.0))]

In [152]:
flags_layer = m.faces.layers.int.new("flags")

In [153]:
f = m.faces[0]

In [154]:
f[flags_layer] = 123

In [158]:
f[m.faces.layers.int["flags"]]

123

In [162]:
f.edges.layers

AttributeError: 'BMElemSeq' object has no attribute 'layers'

In [13]:
bm.Vector((*f.normal, f.verts[0].co @ f.normal))

Vector((-1.0, -0.0, 0.0, 4.0))

In [33]:
m0 = bpy.data.meshes.new("m0")

In [38]:
o0 = bpy.data.objects.new("o0", object_data=m0)

In [40]:
m.to_mesh(m0)

In [44]:
p0 = o0.data.polygons[0]

In [49]:
mm = bmesh.new()
mm.from_mesh(m0)

In [60]:
mm.faces[0][mm.faces.layers.int["flags"]]

123

In [67]:
v = m.verts[0].co

In [72]:
[x.co for x in m.verts[:]]

[Vector((-4.0, -5.0, -5.0)),
 Vector((-4.0, -5.0, 5.0)),
 Vector((-4.0, 5.0, -5.0)),
 Vector((-4.0, 5.0, 5.0)),
 Vector((6.0, -5.0, -5.0)),
 Vector((6.0, -5.0, 5.0)),
 Vector((6.0, 5.0, -5.0)),
 Vector((6.0, 5.0, 5.0)),
 Vector((-4.0, -1.0, 5.0)),
 Vector((-4.0, 5.0, -1.0)),
 Vector((0.0, 5.0, -5.0)),
 Vector((6.0, -1.0, -5.0)),
 Vector((6.0, -5.0, -1.0)),
 Vector((0.0, -5.0, 5.0))]

In [71]:
plane = bm.Vector((1.0, 1.0, 1.0, 0.0))
bmesh.ops.bisect_plane(m, geom=m.faces[:] + m.edges[:] + m.verts[:], 
            plane_co=plane.xyz * plane.w, plane_no=plane.xyz)

{'geom_cut': [<BMVert(0x5646f6fab6f0), index=8>,
  <BMVert(0x5646f6fab728), index=9>,
  <BMVert(0x5646f6fab760), index=10>,
  <BMVert(0x5646f6fab798), index=11>,
  <BMVert(0x5646f6fab7d0), index=12>,
  <BMVert(0x5646f6fab808), index=13>,
  <BMEdge(0x5646f6fb3ab0), index=18, verts=(0x5646f6fab7d0/12, 0x5646f6fab808/13)>,
  <BMEdge(0x5646f6fb3b00), index=19, verts=(0x5646f6fab7d0/12, 0x5646f6fab798/11)>,
  <BMEdge(0x5646f6fb3b50), index=20, verts=(0x5646f6fab760/10, 0x5646f6fab798/11)>,
  <BMEdge(0x5646f6fb3ba0), index=21, verts=(0x5646f6fab728/9, 0x5646f6fab760/10)>,
  <BMEdge(0x5646f6fb3bf0), index=22, verts=(0x5646f6fab6f0/8, 0x5646f6fab728/9)>,
  <BMEdge(0x5646f6fb3c40), index=23, verts=(0x5646f6fab6f0/8, 0x5646f6fab808/13)>],
 'geom': [<BMVert(0x5646f6fab530), index=0>,
  <BMVert(0x5646f6fab568), index=1>,
  <BMVert(0x5646f6fab5a0), index=2>,
  <BMVert(0x5646f6fab5d8), index=3>,
  <BMVert(0x5646f6fab610), index=4>,
  <BMVert(0x5646f6fab648), index=5>,
  <BMVert(0x5646f6fab680), inde

In [7]:
p0 = bm.Vector((1, 1, 1, 1))
p1 = bm.Vector((0, 1, 0, 0))
def pp(p):
    return p.xyz * p.w, p.xyz
bm.geometry.intersect_plane_plane(*pp(p0), *pp(p1))

(Vector((1.5, 0.0, 1.5)),
 Vector((-0.7071067690849304, 0.0, 0.7071067690849304)))

In [21]:
bm.Matrix.OrthoProjection(p0.xyz, 3) @ bm.Vector((2, 2, 2))

Vector((1.1920928955078125e-07, 1.1920928955078125e-07, 1.1920928955078125e-07))

In [3]:
bm.Matrix.Rotation(3.14159256 / 2, 3, 'Z')

Matrix(((7.549790126404332e-08, -1.0, 0.0),
        (1.0, 7.549790126404332e-08, 0.0),
        (0.0, 0.0, 1.0)))

In [16]:
def p(pp):
    ppp = bm.Vector(pp)
    return bm.Vector((*ppp.xy.normalized(), ppp.z / ppp.xy.length))

In [42]:
r = bm.Matrix(((0, -1), (1, 0)))
sgn = bm.Vector((1, -1))
def solve(p0, p1):
    D = (r @ p0.xy) @ p1.xy
    if D < -1e-5:
        pass
    elif D > 1e-5:
        pass
    else:
        return None
    m = bm.Matrix((p0.xy, p1.xy)).transposed()
    d = r @ bm.Vector((p0.z, p1.z))
    Dxy = m @ d
    return Dxy / D * sgn

In [43]:
p0 = p((2, 1, -3))
p1 = p((4, 2, -4))

In [44]:
solve(p0, p1)

In [28]:
m.to_2x2().transposed()

Matrix(((0.8944271802902222, 0.3162277638912201),
        (0.4472135901451111, 0.9486832618713379)))

In [31]:
m.col(2)

TypeError: 'MatrixAccess' object is not callable

In [24]:

def unique_intersection(left, right):
    if len(left) == 0 or len(right) == 0:
        return []
    result = []
    i0 = 0
    i1 = 0
    if left[0] < right[0]:
        rt = right
        right = left
        left = rt

    while i0 < len(left) and i1 < len(right):
        if left[i0] == right[i1]:
            if len(result) == 0 or left[i0] != result[-1]:
                result.append(left[i0])
            i0+=1
        i1+=1
        
    return result

In [26]:
a = [1, 2, 2, 3, 4, 5, 7, 12]
b= [1, 2, 7, 9]
unique_intersection(b, a)

[1, 2, 7]

In [149]:
class leaf:
    def __init__(self, flag):
        self.flag = flag
        
    def add(self, n):
        n.f = leaf(self.flag)
        n.b = leaf(self.flag)
        return n    

    def csg(self, t, op):
        if isinstance(t, leaf):
            self.flag = op(self.flag, t.flag)
            return self
        else:
            res = self.add(t)
            res._f = res._f.csg(t._f, op)
            res._b = res._b.csg(t._b, op)
            return res
    def print(self, d=0):
        return f"{' '*d}leaf: {self.flag}"
        
class node1d:
    def __init__(self, n, d, f, b):
        self.n = n
        self.d = d
    
        self._f = f
        self._b = b
        
    def classify(self, n):
        nn = self.n * n.n
        # dd = n.n * self.d - n.d * self.n
        dd = self.d - nn * n.d
        if dd < -1e-3: return 0x1 # front
        if dd >  1e-3: return 0x2 # back
        if nn > 0.0: return 0x0 # incident
        return 0x4 # opposite
    
    def instance(self):
        return node1d(self.n, self.d, None, None)
    
    def add(self, n):
        c = self.classify(n)
        if c == 0x1:
            self._f = self._f.add(n)
        elif c == 0x2:
            self._b = self._b.add(n)
        return self
        
    def csg(self, t, op):
        if isinstance(t, leaf):
            self._f = self._f.csg(t, op)
            self._b = self._b.csg(t, op)
            return self
        else:
            res = self.add(t)
            res._f = res._f.csg(t._f, op)
            res._b = res._b.csg(t._b, op)
            return res
    def print(self, d=0):
        return f"{' ' * d}{self.n} {self.d}:\n{self._b.print(d + 2)}\n{self._f.print(d + 2)}"

In [143]:
a = node1d(1.0, 1.0, None, None)
b0 = node1d(1.0, 3.0, None, None)
b1 = node1d(-b0.n, -b0.d, None, None)
print(a.classify(b0))
print(a.classify(b1))

1
1


In [150]:
import operator


r0 = node1d(1.0, 1.0, leaf(0), leaf(1))
r1 = node1d(-1.0, 0.0, leaf(0), leaf(1))
s0 = r0.csg(r1, operator.__and__)

r2 = node1d(1.0, 5.0, leaf(0), leaf(1))
r3 = node1d(-1.0, 4.0, leaf(0), leaf(1))
s1 = r2.csg(r3, operator.__and__)

s = s0.csg(s1, operator.__or__)

In [151]:
print(s.print())

1.0 1.0:
  -1.0 0.0:
    leaf: 1
    -1.0 4.0:
      leaf: 1
      leaf: 0
  1.0 5.0:
    -1.0 4.0:
      leaf: 1
      leaf: 0
    leaf: 0
